영업팀 일일보고 자동화 프로그램
===

- 프로그램 제작 목적
    - 인력으로는 매일 꼬박 30분이상 걸리는 작업을 자동화한다.
        - BeFore : 30분 이상 소요
        - After : 40 ~ 50 초 소요
    - 자료 기입 중 실수방지
        - 인력으로 진행하고 단계가 복잡해서 실수의 여지가 꽤 많았다.
    - 작업자별 차이 제거
        - 월 단위 로테이션으로 작업자를 바꿔서 작성한다.
        - 프로그램 제작 중에 알게되었지만 각자 집계기준을 서로 다르게 알고 있다.
            - 수치에 거의 차이가 없으면 상관없으나 차이가 있다.
        - 이 기회에 데이터 집계기준도 재정비하여 통일했다.

- 이 프로그램은 실행파일을 제작해서 영업팀에 배포했으므로 readme.md 파일을 참고할 것

- 브랜치 버전이 존재한다.
    - 최종버전 : sqlalchemy, xlwings 조합
    - pyautogui, win32com, openpyxl 등을 이용한 버전이 있다.
    - 작동에 이상은 없으나 시간단축이 불가능한 단계를 포함하면 자동화해도 10여분 정도 걸린다.

## ERP에서 자료 가져오기

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import binascii   # 한글 변환에 필요한 라이브러리
import sys
from datetime import datetime, date, timedelta
from timeit import default_timer as timer
import pyautogui
import pyperclip # 한글 입력을 위한 클립보드 라이브러리

### DB 접속

In [2]:
sys.path.append('/settings')
import config

def connect_db(sid):
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    
    return conn

engine = connect_db('IVY')

#### 사용할 변수 입력

##### N 시즌 입력

In [3]:
N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

In [4]:
while True:
    i = 0
    if len(N_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if N_season[-1] != 'N':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 N 입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if (N_season[0] >= '0' and N_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')
    
    if (N_season[1] >= '0' and N_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        N_season = pyautogui.prompt(title='N시즌', text='N시즌을 입력하세요. 예 : 23N', default='23N')

    if i == 0:
        break

##### F 시즌 입력

In [5]:
F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

In [6]:
while True:
    i = 0
    if len(F_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if F_season[-1] != 'F':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 F 입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if (F_season[0] >= '0' and F_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')
    
    if (F_season[1] >= '0' and F_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        F_season = pyautogui.prompt(title='F시즌', text='F시즌을 입력하세요. 예 : 22F', default='22F')

    if i == 0:
        break 

##### S 시즌 입력

In [7]:
S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 22S', default='22S')

In [8]:
while True:
    i = 0
    if len(S_season) != 3:
        i += 1
        pyautogui.alert(title='입력오류', text='시즌은 3자리로 입력하세요')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 22S', default='22S')

    if S_season[-1] != 'S':
        i += 1
        pyautogui.alert(title='입력오류', text='마지막 글자는 S 입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 22S', default='22S')

    if (S_season[0] >= '0' and S_season[0] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='첫번째 문자는 숫자입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 22S', default='22S')
    
    if (S_season[1] >= '0' and S_season[1] <= '9') == False:
        i += 1
        pyautogui.alert(title='입력오류', text='두번째 문자는 숫자입니다.')
        S_season = pyautogui.prompt(title='S시즌', text='S시즌을 입력하세요. 예 : 22S', default='22S')

    if i == 0:
        break 

##### 자료 작성기준일 입력

In [9]:
today = datetime.today().strftime("%Y%m%d")
query_date = pyautogui.prompt(title='조회할 날짜', text=f'조회할 날짜를 입력하세요. 예 : {today}', default=today)

##### 보고일자 계산

In [10]:
# 작성일자(조회일자)가 금요일이면 다음주 월요일이 보고일자
if datetime.strptime(query_date, '%Y%m%d').weekday() == 4:
    report_date = (datetime.strptime(query_date, '%Y%m%d') + timedelta(days=3)).strftime("%Y%m%d")
else:
    report_date = (datetime.strptime(query_date, '%Y%m%d') + timedelta(days=1)).strftime("%Y%m%d")
    
report_date

'20220620'

In [11]:
while True:
    i = 0
    if len(query_date) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        query_date = pyautogui.prompt(title='조회할 날짜', text=f'조회할 날짜를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

##### 주간조회 시작일자 입력

In [12]:
# 지난주 토요일 구하기

diff = (date.today().weekday() - 5) % 7 # 0:월요일, 1:화요일, 2:수요일, 3:목요일, 4:금요일, 5:토요일, 6:일요일
last_sat = (date.today() - timedelta(days=diff)).strftime("%Y%m%d") # 대쉬를 뺀 형식으로 변경
print(last_sat)

20220611


In [13]:
week_start = pyautogui.prompt(title='주간조회 시작날짜', text=f'주간조회 시작날짜를 입력하세요. 예 : {last_sat}', default=last_sat)

In [14]:
while True:
    i = 0
    if len(week_start) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        week_start = pyautogui.prompt(title='주간조회 시작날짜', text=f'주간조회 시작날짜를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

##### 주간조회 종료일자 입력

In [15]:
week_end = pyautogui.prompt(title='주간조회 종료날짜', text=f'주간조회 종료날짜를 입력하세요. 예 : {today}', default=today)

In [16]:
while True:
    i = 0
    if len(week_end) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        week_end = pyautogui.prompt(title='주간조회 종료날짜', text=f'주간조회 종료날짜를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

In [17]:
confirm = pyautogui.confirm(title='알림', text='작업을 시작합니다.\n취소하면 작업을 중단합니다.')
confirm

if confirm == 'Cancel':
    sys.exit()

### 수행시간측정 시작

In [18]:
start_lab = timer()

In [19]:
# 나중에 사용할 수도.

etc_cust = '' # 일반업체
g2b_gb = '' # 입찰구분


"""
< 일반낙찰업체 리스트 (etc_cust 에 기입) >

A2  미치코런던
A3	세이트스코트
A4	에이스베이직
A5	프리모
A6	스쿨뱅크
A7	하이틴
A8	런던베이직
A9	이엠씨
B1	김설영
B2	아이니
B4	쎈텐
Z	기타


< 입찰구분 (g2b_gb 에 기입) >
G G2B
1 수의계약
2 협의구매
"""

if etc_cust == '':
    etc = ''
else:
    etc = f" and a.g2b_co_gb  in ('E','I','L','S','{etc_cust}') "

if g2b_gb == '':
    g2b = ''
else:
    g2b = f" and a.g2b_gb  = '{g2b_gb}' "

### SQL문

In [20]:
# with open("파일이름.sql", "rt", encoding="utf8") as f:
#     oracleSql = f.read()

# 길이가 길 경우 파일로 넘겨준다.

# oracleSql = '''
# select agen_code,
#        rawtohex(utl_raw.cast_to_raw(agen_name)) agen_name
# from i_agen_t
# where 1=1
# '''

#### 한글변환 함수
    - SQL문의 한글 컬럼에는 다음과 같이 함수로 처리
    - rawtohex(utl_raw.cast_to_raw(변환할 컬럼))

In [21]:
# US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df):
    for index, byte_data in enumerate(df):
        # print(byte_data + "  >>>  ", end="")
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
        # print(df[index])
    return df

#### 전자입찰현황 (특약별) 

In [22]:
def bid_tkyk(query_dt, season1, season2, etc_sql, g2b_sql):
    j_query_dt = str(int(query_dt[:4])-1) + query_dt[4:] # 1년전
    j_season1 = str(int(season1[0:2])-1) + season1[-1] # 지난시즌
    if len(season2) == 3:
        j_season2 = str(int(season2[0:2])-1) + season2[-1]
    else:
        j_season2 = ''
    
    oracleSql = f'''
SELECT   z.tkyk,
         Rawtohex(utl_raw.Cast_to_raw(tkyk_name)) tkyk_name,
         sort,
         SUM(z.is_cnt)        is_cnt,
         SUM(z.is_cnt2)       is_cnt2,
         SUM(z.i_cnt)         i_cnt,
         SUM(z.s_cnt)         s_cnt,
         SUM(z.e_cnt)         e_cnt,
         SUM(z.l_cnt)         l_cnt,
         SUM(z.etc_cnt)       etc_cnt,
         SUM(z.i_qty)         i_qty,
         SUM(z.s_qty)         s_qty,
         SUM(z.e_qty)         e_qty,
         SUM(z.l_qty)         l_qty,
         SUM(z.etc_qty)       etc_qty,
         SUM(z.f_tot_cnt)     f_tot_cnt,
         SUM(z.f_sch_cnt)     f_sch_cnt,
         SUM(z.f_end_amt)     f_end_amt,
         SUM(z.f_stand_amt)   f_stand_amt,
         SUM(z.f_i_sch_cnt)   f_i_sch_cnt,
         SUM(z.s_tot_cnt)     s_tot_cnt,
         SUM(z.s_sch_cnt)     s_sch_cnt,
         SUM(z.s_end_amt)     s_end_amt,
         SUM(z.s_stand_amt)   s_stand_amt,
         SUM(z.s_i_sch_cnt)   s_i_sch_cnt,
         SUM(z.j_is_cnt)      j_is_cnt,
         SUM(z.j_is_cnt2)     j_is_cnt2,
         SUM(z.j_i_cnt)       j_i_cnt,
         SUM(z.j_s_cnt)       j_s_cnt,
         SUM(z.j_e_cnt)       j_e_cnt,
         SUM(z.j_l_cnt)       j_l_cnt,
         SUM(z.j_etc_cnt)     j_etc_cnt,
         SUM(z.j_i_qty)       j_i_qty,
         SUM(z.j_s_qty)       j_s_qty,
         SUM(z.j_e_qty)       j_e_qty,
         SUM(z.j_l_qty)       j_l_qty,
         SUM(z.j_etc_qty)     j_etc_qty,
         SUM(z.j_f_tot_cnt)   j_f_tot_cnt,
         SUM(z.j_f_sch_cnt)   j_f_sch_cnt,
         SUM(z.j_f_end_amt)   j_f_end_amt,
         SUM(z.j_f_stand_amt) j_f_stand_amt,
         SUM(z.j_f_i_sch_cnt) j_f_i_sch_cnt,
         SUM(z.j_s_tot_cnt)   j_s_tot_cnt,
         SUM(z.j_s_sch_cnt)   j_s_sch_cnt,
         SUM(z.j_s_end_amt)   j_s_end_amt,
         SUM(z.j_s_stand_amt) j_s_stand_amt,
         SUM(z.j_s_i_sch_cnt) j_s_i_sch_cnt
FROM     (
                SELECT a.g2b_tkyk                      tkyk,
                       To_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       Decode(a.g2b_sch_gb,
                              'C', 2,
                              1) is_cnt2,
                       Decode(a.g2b_co_gb,
                              'I', 1,
                              0) i_cnt,
                       Decode(a.g2b_co_gb,
                              'E', 1,
                              0) e_cnt,
                       Decode(a.g2b_co_gb,
                              'S', 1,
                              0) s_cnt,
                       Decode(a.g2b_co_gb,
                              'L', 1,
                              0) l_cnt,
                       Decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              1) etc_cnt,
                       Decode(a.g2b_co_gb,
                              'I', g2b_qty,
                              0) i_qty,
                       Decode(a.g2b_co_gb,
                              'E', g2b_qty,
                              0) e_qty,
                       Decode(a.g2b_co_gb,
                              'S', g2b_qty,
                              0) s_qty,
                       Decode(a.g2b_co_gb,
                              'L', g2b_qty,
                              0) l_qty,
                       Decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              g2b_qty) etc_qty,
                       0               f_tot_cnt,
                       0               f_sch_cnt,
                       0               f_end_amt,
                       0               f_stand_amt,
                       0               f_i_sch_cnt,
                       0               s_tot_cnt,
                       0               s_sch_cnt,
                       0               s_end_amt,
                       0               s_stand_amt,
                       0               s_i_sch_cnt,
                       0               j_is_cnt,
                       0               j_is_cnt2,
                       0               j_i_cnt,
                       0               j_e_cnt,
                       0               j_s_cnt,
                       0               j_l_cnt,
                       0               j_etc_cnt,
                       0               j_i_qty,
                       0               j_e_qty,
                       0               j_s_qty,
                       0               j_l_qty,
                       0               j_etc_qty,
                       0               j_f_tot_cnt,
                       0               j_f_sch_cnt,
                       0               j_f_end_amt,
                       0               j_f_stand_amt,
                       0               j_f_i_sch_cnt,
                       0               j_s_tot_cnt,
                       0               j_s_sch_cnt,
                       0               j_s_end_amt,
                       0               j_s_stand_amt,
                       0               j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= To_date('{query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{season1}',
                                               '{season2}' )
                       OR     a.g2b_quota2 IN ( '{season1}',
                                               '{season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk                      tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       1                               f_tot_cnt,
                       a.g2b_qty                       f_sch_cnt,
                       a.g2b_end_amt                   f_end_amt,
                       a.g2b_stand_amt                 f_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) f_i_sch_cnt,
                       0         s_tot_cnt,
                       0         s_sch_cnt,
                       0         s_end_amt,
                       0         s_stand_amt,
                       0         s_i_sch_cnt,
                       0         j_is_cnt,
                       0         j_is_cnt2,
                       0         j_i_cnt,
                       0         j_e_cnt,
                       0         j_s_cnt,
                       0         j_l_cnt,
                       0         j_etc_cnt,
                       0         j_i_qty,
                       0         j_e_qty,
                       0         j_s_qty,
                       0         j_l_qty,
                       0         j_etc_qty,
                       0         j_f_tot_cnt,
                       0         j_f_sch_cnt,
                       0         j_f_end_amt,
                       0         j_f_stand_amt,
                       0         j_f_i_sch_cnt,
                       0         j_s_tot_cnt,
                       0         j_s_sch_cnt,
                       0         j_s_end_amt,
                       0         j_s_stand_amt,
                       0         j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'N',
                                        'F',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{season1}',
                                               '{season2}' )
                       OR     a.g2b_quota2 IN ( '{season1}',
                                               '{season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       1                               s_tot_cnt,
                       a.g2b_qty                       s_sch_cnt,
                       a.g2b_end_amt                   s_end_amt,
                       a.g2b_stand_amt                 s_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) s_i_sch_cnt,
                       0         j_is_cnt,
                       0         j_is_cnt2,
                       0         j_i_cnt,
                       0         j_e_cnt,
                       0         j_s_cnt,
                       0         j_l_cnt,
                       0         j_etc_cnt,
                       0         j_i_qty,
                       0         j_e_qty,
                       0         j_s_qty,
                       0         j_l_qty,
                       0         j_etc_qty,
                       0         j_f_tot_cnt,
                       0         j_f_sch_cnt,
                       0         j_f_end_amt,
                       0         j_f_stand_amt,
                       0         j_f_i_sch_cnt,
                       0         j_s_tot_cnt,
                       0         j_s_sch_cnt,
                       0         j_s_end_amt,
                       0         j_s_stand_amt,
                       0         j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'S',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{season1}',
                                               '{season2}' )
                       OR     a.g2b_quota2 IN ( '{season1}',
                                               '{season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk                      tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       0                               s_tot_cnt,
                       0                               s_sch_cnt,
                       0                               s_end_amt,
                       0                               s_stand_amt,
                       0                               s_i_sch_cnt,
                       0                               j_is_cnt,
                       decode(a.g2b_sch_gb,
                              'C', 2,
                              1) j_is_cnt2,
                       decode(a.g2b_co_gb,
                              'I', 1,
                              0) j_i_cnt,
                       decode(a.g2b_co_gb,
                              'E', 1,
                              0) j_e_cnt,
                       decode(a.g2b_co_gb,
                              'S', 1,
                              0) j_s_cnt,
                       decode(a.g2b_co_gb,
                              'L', 1,
                              0) j_l_cnt,
                       decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              1) j_etc_cnt,
                       decode(a.g2b_co_gb,
                              'I', g2b_qty,
                              0) j_i_qty,
                       decode(a.g2b_co_gb,
                              'E', g2b_qty,
                              0) j_e_qty,
                       decode(a.g2b_co_gb,
                              'S', g2b_qty,
                              0) j_s_qty,
                       decode(a.g2b_co_gb,
                              'L', g2b_qty,
                              0) j_l_qty,
                       decode(a.g2b_co_gb,
                              'I', 0,
                              'E', 0,
                              'S', 0,
                              'L', 0,
                              g2b_qty) j_etc_qty,
                       0               j_f_tot_cnt,
                       0               j_f_sch_cnt,
                       0               j_f_end_amt,
                       0               j_f_stand_amt,
                       0               j_f_i_sch_cnt,
                       0               j_s_tot_cnt,
                       0               j_s_sch_cnt,
                       0               j_s_end_amt,
                       0               vs_stand_amt,
                       0               j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{j_query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{j_season1}',
                                               '{j_season2}' )
                       OR     a.g2b_quota2 IN ( '{j_season1}',
                                               '{j_season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk                      tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       0                               s_tot_cnt,
                       0                               s_sch_cnt,
                       0                               s_end_amt,
                       0                               s_stand_amt,
                       0                               s_i_sch_cnt,
                       0                               j_is_cnt,
                       0                               j_is_cnt2,
                       0                               j_i_cnt,
                       0                               j_e_cnt,
                       0                               j_s_cnt,
                       0                               j_l_cnt,
                       0                               j_etc_cnt,
                       0                               j_i_qty,
                       0                               j_e_qty,
                       0                               j_s_qty,
                       0                               j_l_qty,
                       0                               j_etc_qty,
                       1                               j_f_tot_cnt,
                       a.g2b_qty                       j_f_sch_cnt,
                       a.g2b_end_amt                   j_f_end_amt,
                       a.g2b_stand_amt                 j_f_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) j_f_i_sch_cnt,
                       0         j_s_tot_cnt,
                       0         j_s_sch_cnt,
                       0         j_s_end_amt,
                       0         j_s_stand_amt,
                       0         j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'N',
                                        'F',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{j_query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{j_season1}',
                                               '{j_season2}' )
                       OR     a.g2b_quota2 IN ( '{j_season1}',
                                               '{j_season2}' ) ) {etc_sql} {g2b_sql}
                UNION ALL
                SELECT a.g2b_tkyk,
                       to_char(a.g2b_date, 'yyyymmdd') g2b_date,
                       0                               is_cnt,
                       0                               is_cnt2,
                       0                               i_cnt,
                       0                               e_cnt,
                       0                               s_cnt,
                       0                               l_cnt,
                       0                               etc_cnt,
                       0                               i_qty,
                       0                               e_qty,
                       0                               s_qty,
                       0                               l_qty,
                       0                               etc_qty,
                       0                               f_tot_cnt,
                       0                               f_sch_cnt,
                       0                               f_end_amt,
                       0                               f_stand_amt,
                       0                               f_i_sch_cnt,
                       0                               s_tot_cnt,
                       0                               s_sch_cnt,
                       0                               s_end_amt,
                       0                               s_stand_amt,
                       0                               s_i_sch_cnt,
                       0                               j_is_cnt,
                       0                               j_is_cnt2,
                       0                               j_i_cnt,
                       0                               j_e_cnt,
                       0                               j_s_cnt,
                       0                               j_l_cnt,
                       0                               j_etc_cnt,
                       0                               j_i_qty,
                       0                               j_e_qty,
                       0                               j_s_qty,
                       0                               j_l_qty,
                       0                               j_etc_qty,
                       0                               j_f_tot_cnt,
                       0                               j_f_sch_cnt,
                       0                               j_f_end_amt,
                       0                               j_f_stand_amt,
                       0                               j_f_i_sch_cnt,
                       1                               j_s_tot_cnt,
                       a.g2b_qty                       j_s_sch_cnt,
                       a.g2b_end_amt                   j_s_end_amt,
                       a.g2b_stand_amt                 j_s_stand_amt,
                       decode(a.g2b_co_gb,
                              'I', a.g2b_qty,
                              0) j_s_i_sch_cnt
                FROM   i_sale_g2b_t a,
                       i_sch_com_t
                WHERE  schc_code (+) = a.g2b_school
                AND    a.g2b_season IN ( 'S',
                                        'A',
                                        'B',
                                        'C' )
                AND    a.g2b_end_gb = '9'
                AND    a.g2b_date <= to_date('{j_query_dt}', 'yyyymmdd')
                AND    (
                              a.g2b_quota1 IN ( '{j_season1}',
                                               '{j_season2}' )
                       OR     a.g2b_quota2 IN ( '{j_season1}',
                                               '{j_season2}' ) ) {etc_sql} {g2b_sql} ) z,
         i_tkyk_t
WHERE    z.tkyk = tkyk_code
GROUP BY z.tkyk,
         tkyk_name,
         SORT
ORDER BY SORT
'''
    df = pd.read_sql_query(oracleSql, engine)
    df_temp = df['tkyk_name'].copy()
    df['tkyk_name'] = us7ascii_to_cp949(df_temp)
    
    df['quota'] = f'{season1}{season2}'
    df = df[['quota', 'tkyk', 'sort', 'tkyk_name', 'i_cnt', 's_cnt', 'e_cnt', 'l_cnt', 'etc_cnt', 'i_qty', 's_qty', 'e_qty', 'l_qty', 'etc_qty']]
    return df

##### 23N22F

In [23]:
df_bid_tkyk_23N22F = bid_tkyk(query_date, N_season, F_season, etc, g2b)
df_bid_tkyk_23N22F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,1,1,0,0,0,140,153,0,0,0
1,23N22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,23N22F,D,3,대전상권,1,1,0,0,0,149,184,0,0,0
3,23N22F,L,4,광주상권,49,49,39,48,60,7247,6688,5443,6678,8455
4,23N22F,I,5,대구상권,41,34,36,26,42,6426,5066,5709,4319,7752
5,23N22F,R,6,부산상권,22,7,10,8,17,2137,707,1225,1169,2974


##### 없는 상권 추가하기 (함수)

In [24]:
tkyk_name = ['서울상권', '중부상권', '대전상권', '광주상권', '대구상권', '부산상권']
tkyk_code = ['C', 'H', 'D', 'L', 'I', 'R']
tkyk_sort = [1, 2, 3, 4, 5, 6]

def add_tkyk(df):
    tkyk_lost = list(set(tkyk_sort) - set(df['sort'].tolist()) )
    for t in tkyk_lost:
        tkyk_insert = []
        tkyk_insert.append(df['quota'][0])
        tkyk_insert.append(tkyk_code[t-1])
        tkyk_insert.append(tkyk_sort[t-1])
        tkyk_insert.append(tkyk_name[t-1])
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        tkyk_insert.append(0)
        df.loc[len(df)] = tkyk_insert
    df = df.sort_values('sort').reset_index(drop=True)
    return df

In [25]:
df_bid_tkyk_23N22F = add_tkyk(df_bid_tkyk_23N22F)
df_bid_tkyk_23N22F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,1,1,0,0,0,140,153,0,0,0
1,23N22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,23N22F,D,3,대전상권,1,1,0,0,0,149,184,0,0,0
3,23N22F,L,4,광주상권,49,49,39,48,60,7247,6688,5443,6678,8455
4,23N22F,I,5,대구상권,41,34,36,26,42,6426,5066,5709,4319,7752
5,23N22F,R,6,부산상권,22,7,10,8,17,2137,707,1225,1169,2974


##### 붙여넣을 합계 데이터

In [26]:
df_bid_tkyk_23N22F_ISEL_sum = df_bid_tkyk_23N22F[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty']].copy()
df_bid_tkyk_23N22F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,1,140,1,153,0,0,0,0
1,37,4762,37,5387,23,3353,25,4246
2,1,149,1,184,0,0,0,0
3,49,7247,49,6688,39,5443,48,6678
4,41,6426,34,5066,36,5709,26,4319
5,22,2137,7,707,10,1225,8,1169


In [27]:
df_bid_tkyk_23N22F_ISEL_sum['sum_cnt'] = (df_bid_tkyk_23N22F_ISEL_sum['i_cnt'] +
                                          df_bid_tkyk_23N22F_ISEL_sum['s_cnt'] +
                                          df_bid_tkyk_23N22F_ISEL_sum['e_cnt'] +
                                          df_bid_tkyk_23N22F_ISEL_sum['l_cnt'])

df_bid_tkyk_23N22F_ISEL_sum['sum_qty'] = (df_bid_tkyk_23N22F_ISEL_sum['i_qty'] +
                                          df_bid_tkyk_23N22F_ISEL_sum['s_qty'] +
                                          df_bid_tkyk_23N22F_ISEL_sum['e_qty'] +
                                          df_bid_tkyk_23N22F_ISEL_sum['l_qty'])
df_bid_tkyk_23N22F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,1,140,1,153,0,0,0,0,2,293
1,37,4762,37,5387,23,3353,25,4246,122,17748
2,1,149,1,184,0,0,0,0,2,333
3,49,7247,49,6688,39,5443,48,6678,185,26056
4,41,6426,34,5066,36,5709,26,4319,137,21520
5,22,2137,7,707,10,1225,8,1169,47,5238


In [28]:
df_bid_tkyk_23N22F_ISEL_sum.loc[len(df_bid_tkyk_23N22F_ISEL_sum)] = df_bid_tkyk_23N22F_ISEL_sum.sum()
df_bid_tkyk_23N22F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,1,140,1,153,0,0,0,0,2,293
1,37,4762,37,5387,23,3353,25,4246,122,17748
2,1,149,1,184,0,0,0,0,2,333
3,49,7247,49,6688,39,5443,48,6678,185,26056
4,41,6426,34,5066,36,5709,26,4319,137,21520
5,22,2137,7,707,10,1225,8,1169,47,5238
6,151,20861,129,18185,108,15730,107,16412,495,71188


##### 22F

In [29]:
df_bid_tkyk_22F = bid_tkyk(query_date, F_season, '', etc, g2b)
df_bid_tkyk_22F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22F,C,1,서울상권,1,0,0,0,0,140,0,0,0,0
1,22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,22F,D,3,대전상권,0,0,0,0,0,0,0,0,0,0
3,22F,L,4,광주상권,47,49,38,47,59,6776,6688,5243,6422,8065
4,22F,I,5,대구상권,41,34,36,26,42,6426,5066,5709,4319,7752
5,22F,R,6,부산상권,22,7,10,8,16,2137,707,1225,1169,2818


In [30]:
df_bid_tkyk_22F = add_tkyk(df_bid_tkyk_22F)
df_bid_tkyk_22F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22F,C,1,서울상권,1,0,0,0,0,140,0,0,0,0
1,22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,22F,D,3,대전상권,0,0,0,0,0,0,0,0,0,0
3,22F,L,4,광주상권,47,49,38,47,59,6776,6688,5243,6422,8065
4,22F,I,5,대구상권,41,34,36,26,42,6426,5066,5709,4319,7752
5,22F,R,6,부산상권,22,7,10,8,16,2137,707,1225,1169,2818


##### 붙여넣을 합계 데이터

In [31]:
df_bid_tkyk_22F_ISEL_sum = df_bid_tkyk_22F[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty']].copy()
df_bid_tkyk_22F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,1,140,0,0,0,0,0,0
1,37,4762,37,5387,23,3353,25,4246
2,0,0,0,0,0,0,0,0
3,47,6776,49,6688,38,5243,47,6422
4,41,6426,34,5066,36,5709,26,4319
5,22,2137,7,707,10,1225,8,1169


In [32]:
# 액샐에서 처리

# df_bid_tkyk_22F_ISEL_sum['sum_cnt'] = (df_bid_tkyk_22F_ISEL_sum['i_cnt'] +
#                                        df_bid_tkyk_22F_ISEL_sum['s_cnt'] +
#                                        df_bid_tkyk_22F_ISEL_sum['e_cnt'] +
#                                        df_bid_tkyk_22F_ISEL_sum['l_cnt'])

# df_bid_tkyk_22F_ISEL_sum['sum_qty'] = (df_bid_tkyk_22F_ISEL_sum['i_qty'] +
#                                        df_bid_tkyk_22F_ISEL_sum['s_qty'] +
#                                        df_bid_tkyk_22F_ISEL_sum['e_qty'] +
#                                        df_bid_tkyk_22F_ISEL_sum['l_qty'])
# df_bid_tkyk_22F_ISEL_sum

In [33]:
df_bid_tkyk_22F_ISEL_sum.loc[len(df_bid_tkyk_22F_ISEL_sum)] = df_bid_tkyk_22F_ISEL_sum.sum()
df_bid_tkyk_22F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,1,140,0,0,0,0,0,0
1,37,4762,37,5387,23,3353,25,4246
2,0,0,0,0,0,0,0,0
3,47,6776,49,6688,38,5243,47,6422
4,41,6426,34,5066,36,5709,26,4319
5,22,2137,7,707,10,1225,8,1169
6,148,20241,127,17848,107,15530,106,16156


##### 22S

In [34]:
df_bid_tkyk_22S = bid_tkyk(query_date, S_season, '', etc, g2b)
df_bid_tkyk_22S

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22S,C,1,서울상권,171,167,127,127,386,33270,32474,24909,25254,76495
1,22S,H,2,중부상권,223,217,175,194,314,45731,48693,36576,43285,68225
2,22S,D,3,대전상권,161,125,109,95,153,22894,19628,16711,15137,20120
3,22S,L,4,광주상권,121,115,101,99,127,18088,17524,16023,15160,20081
4,22S,I,5,대구상권,103,103,73,70,96,18112,17639,12023,12083,17966
5,22S,R,6,부산상권,144,87,82,87,116,22038,14568,13419,14600,18879


In [35]:
df_bid_tkyk_22S = add_tkyk(df_bid_tkyk_22S)
df_bid_tkyk_22S

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22S,C,1,서울상권,171,167,127,127,386,33270,32474,24909,25254,76495
1,22S,H,2,중부상권,223,217,175,194,314,45731,48693,36576,43285,68225
2,22S,D,3,대전상권,161,125,109,95,153,22894,19628,16711,15137,20120
3,22S,L,4,광주상권,121,115,101,99,127,18088,17524,16023,15160,20081
4,22S,I,5,대구상권,103,103,73,70,96,18112,17639,12023,12083,17966
5,22S,R,6,부산상권,144,87,82,87,116,22038,14568,13419,14600,18879


##### 붙여넣을 합계 데이터

In [36]:
df_bid_tkyk_22S_ISEL_sum = df_bid_tkyk_22S[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty']].copy()
df_bid_tkyk_22S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,171,33270,167,32474,127,24909,127,25254
1,223,45731,217,48693,175,36576,194,43285
2,161,22894,125,19628,109,16711,95,15137
3,121,18088,115,17524,101,16023,99,15160
4,103,18112,103,17639,73,12023,70,12083
5,144,22038,87,14568,82,13419,87,14600


In [37]:
df_bid_tkyk_22S_ISEL_sum['sum_cnt'] = (df_bid_tkyk_22S_ISEL_sum['i_cnt'] +
                                       df_bid_tkyk_22S_ISEL_sum['s_cnt'] +
                                       df_bid_tkyk_22S_ISEL_sum['e_cnt'] +
                                       df_bid_tkyk_22S_ISEL_sum['l_cnt'])

df_bid_tkyk_22S_ISEL_sum['sum_qty'] = (df_bid_tkyk_22S_ISEL_sum['i_qty'] +
                                       df_bid_tkyk_22S_ISEL_sum['s_qty'] +
                                       df_bid_tkyk_22S_ISEL_sum['e_qty'] +
                                       df_bid_tkyk_22S_ISEL_sum['l_qty'])
df_bid_tkyk_22S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,171,33270,167,32474,127,24909,127,25254,592,115907
1,223,45731,217,48693,175,36576,194,43285,809,174285
2,161,22894,125,19628,109,16711,95,15137,490,74370
3,121,18088,115,17524,101,16023,99,15160,436,66795
4,103,18112,103,17639,73,12023,70,12083,349,59857
5,144,22038,87,14568,82,13419,87,14600,400,64625


In [38]:
df_bid_tkyk_22S_ISEL_sum.loc[len(df_bid_tkyk_22S_ISEL_sum)] = df_bid_tkyk_22S_ISEL_sum.sum()
df_bid_tkyk_22S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,171,33270,167,32474,127,24909,127,25254,592,115907
1,223,45731,217,48693,175,36576,194,43285,809,174285
2,161,22894,125,19628,109,16711,95,15137,490,74370
3,121,18088,115,17524,101,16023,99,15160,436,66795
4,103,18112,103,17639,73,12023,70,12083,349,59857
5,144,22038,87,14568,82,13419,87,14600,400,64625
6,923,160133,814,150526,667,119661,672,125519,3076,555839


##### 붙여넣을 합계 테이터 (M.S)

In [39]:
df_bid_tkyk_23N22F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,1,1,0,0,0,140,153,0,0,0
1,23N22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,23N22F,D,3,대전상권,1,1,0,0,0,149,184,0,0,0
3,23N22F,L,4,광주상권,49,49,39,48,60,7247,6688,5443,6678,8455
4,23N22F,I,5,대구상권,41,34,36,26,42,6426,5066,5709,4319,7752
5,23N22F,R,6,부산상권,22,7,10,8,17,2137,707,1225,1169,2974


In [40]:
df_ms_total = pd.DataFrame()
df_ms_total = pd.concat([df_ms_total,
                         df_bid_tkyk_23N22F.groupby('quota')[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty', 'etc_cnt', 'etc_qty']].agg(sum).T,
                         df_bid_tkyk_22F.groupby('quota')[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty', 'etc_cnt', 'etc_qty']].agg(sum).T,
                         df_bid_tkyk_22S.groupby('quota')[['i_cnt', 'i_qty', 's_cnt', 's_qty', 'e_cnt', 'e_qty', 'l_cnt', 'l_qty', 'etc_cnt', 'etc_qty']].agg(sum).T
                        ], axis=1)
df_ms_total['23N'] = df_ms_total['23N22F'] - df_ms_total['22F']

# 집계기준용 컬럼
df_ms_total = df_ms_total.reset_index()
df_ms_total[['gbn1', 'gbn2']] = df_ms_total['index'].str.split('_', n=1, expand=True)

# 합계라인 추가
df_ms_total = pd.concat([df_ms_total, df_ms_total.groupby('gbn2')[['23N22F', '22F', '22S', '23N']].agg(sum)])

df_ms_total

quota,index,23N22F,22F,22S,23N,gbn1,gbn2
0,i_cnt,151,148,923,3,i,cnt
1,i_qty,20861,20241,160133,620,i,qty
2,s_cnt,129,127,814,2,s,cnt
3,s_qty,18185,17848,150526,337,s,qty
4,e_cnt,108,107,667,1,e,cnt
5,e_qty,15730,15530,119661,200,e,qty
6,l_cnt,107,106,672,1,l,cnt
7,l_qty,16412,16156,125519,256,l,qty
8,etc_cnt,147,145,1192,2,etc,cnt
9,etc_qty,22971,22425,221766,546,etc,qty


In [41]:
i_lst = ['',
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], '23N22F'] / df_ms_total.loc['qty', '23N22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], '22F'] / df_ms_total.loc['qty', '22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], '22S'] / df_ms_total.loc['qty', '22S'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0], '23N'] / df_ms_total.loc['qty', '23N'], 3),
         'i',
         ''
        ]

s_lst = ['',
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], '23N22F'] / df_ms_total.loc['qty', '23N22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], '22F'] / df_ms_total.loc['qty', '22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], '22S'] / df_ms_total.loc['qty', '22S'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0], '23N'] / df_ms_total.loc['qty', '23N'], 3),
         's',
         ''
        ]

e_lst = ['',
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], '23N22F'] / df_ms_total.loc['qty', '23N22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], '22F'] / df_ms_total.loc['qty', '22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], '22S'] / df_ms_total.loc['qty', '22S'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0], '23N'] / df_ms_total.loc['qty', '23N'], 3),
         'e',
         ''
        ]

l_lst = ['',
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], '23N22F'] / df_ms_total.loc['qty', '23N22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], '22F'] / df_ms_total.loc['qty', '22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], '22S'] / df_ms_total.loc['qty', '22S'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0], '23N'] / df_ms_total.loc['qty', '23N'], 3),
         'l',
         ''
        ]

etc_lst = ['',
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], '23N22F'] / df_ms_total.loc['qty', '23N22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], '22F'] / df_ms_total.loc['qty', '22F'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], '22S'] / df_ms_total.loc['qty', '22S'], 3),
         round(df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0], '23N'] / df_ms_total.loc['qty', '23N'], 3),
         'etc',
         ''
        ]

i_lst

['', 0.222, 0.22, 0.206, 0.316, 'i', '']

In [42]:
df_ms_total = df_ms_total.reset_index(drop=True)

df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'i_qty'].index[0]+0.5] = i_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 's_qty'].index[0]+0.5] = s_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'e_qty'].index[0]+0.5] = e_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'l_qty'].index[0]+0.5] = l_lst
df_ms_total.loc[df_ms_total[df_ms_total['index'] == 'etc_qty'].index[0]+0.5] = etc_lst

df_ms_total = df_ms_total.sort_index().reset_index(drop=True)

df_ms_total = df_ms_total[['23N22F', '23N', '22S', '22F']]

df_ms_total

quota,23N22F,23N,22S,22F
0,151.000,3.000,923.000,148.000
1,20861.000,620.000,160133.000,20241.000
2,0.222,0.316,0.206,0.220
3,129.000,2.000,814.000,127.000
4,18185.000,337.000,150526.000,17848.000
5,0.193,0.172,0.194,0.194
6,108.000,1.000,667.000,107.000
7,15730.000,200.000,119661.000,15530.000
8,0.167,0.102,0.154,0.168
9,107.000,1.000,672.000,106.000


#### 수주량 조회 (특약별) 

In [43]:
def suju_query(season1, season2, season3):
    oracleSql = f'''
SELECT z.QUOTA,
       z.gubun,
       z.bokjong,
       z.tkyk,
       Rawtohex(utl_raw.Cast_to_raw(tkyk_name)) tkyk_name,
       sort,
       SUM(z.qty)                               qty
FROM   (SELECT '{season1}{season2}' QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season1}{season2}' QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season1}{season2}' QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season1}{season2}' QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season1}', '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '03'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_sch_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'F'
               AND master_bokjong = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
               AND sch_code(+) = master_school
               AND master_bokjong = sch_f_bokjong
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season2}'          QUOTA,
               '12'                 gubun,
               Max(master_bokjong)  bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t
        WHERE  master_quota IN ( '{season2}' )
               AND master_jaepum = 'H'
               AND master_bokjong = 'J'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season3}'          QUOTA,
               '03'                 gubun,
               'HA'                 bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_cod_t tt
        WHERE  master_quota IN ( '{season3}' )
               AND master_jaepum = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '03'
               AND master_status <= '60'
               AND tt.cod_code = master_bokjong
               AND tt.cod_gbn_code = '01'
               AND Nvl(tt.cod_etc2, '') = 'H'
               AND Nvl(tt.cod_etc3, '') IN ( 'S', 'T' )
        GROUP  BY master_tkyk
        UNION ALL
        SELECT '{season3}'          QUOTA,
               '12'                 gubun,
               'HA'                 bokjong,
               master_tkyk          tkyk,
               SUM(master_suju_qty) qty
        FROM   i_suju_master_t,
               i_cod_t tt
        WHERE  master_quota IN ( '{season3}' )
               AND master_jaepum = 'H'
               AND master_tkyk IN ( 'C', 'D', 'H', 'I',
                                    'L', 'R' )
               AND master_status >= '12'
               AND master_status <= '60'
               AND tt.cod_code = master_bokjong
               AND tt.cod_gbn_code = '01'
               AND Nvl(tt.cod_etc2, '') = 'H'
               AND Nvl(tt.cod_etc3, '') IN ( 'S', 'T' )
        GROUP  BY master_tkyk) z,
       i_tkyk_t
WHERE  z.tkyk = tkyk_code
GROUP  BY z.QUOTA,
          z.gubun,
          z.bokjong,
          z.tkyk,
          tkyk_name,
          sort
ORDER  BY z.QUOTA,
          z.gubun,
          z.bokjong DESC,
          sort
'''
    df = pd.read_sql_query(oracleSql, engine)
    df_temp = df['tkyk_name'].copy()
    df['tkyk_name'] = us7ascii_to_cp949(df_temp)
    
    df['gubun'] = df['gubun'].str.replace('03', '수주량').str.replace('12', '해제량')
    return df

In [44]:
df_suju_qty = suju_query(N_season, F_season, S_season)
df_suju_qty.head()

,quota,gubun,bokjong,tkyk,tkyk_name,sort,qty
0,22F,수주량,J,C,서울상권,1,115
1,22F,수주량,J,H,중부상권,2,4652
2,22F,수주량,J,D,대전상권,3,54
3,22F,수주량,J,L,광주상권,4,3403
4,22F,수주량,J,I,대구상권,5,4643


#### 수주량 전주간 조회 (수주일 기준 / 영업확정일 기준)

     - ST는 03 ~ 60 으로 고정
     - 동복은 복종 J 기준, 하복은 하의 기준

In [45]:
def suju_query_week(season1, season2, season3, start_dt, end_dt):
    oracleSql = f'''
SELECT z.quota,
       z.gubun,
       z.bokjong,
       Sum(z.qty) qty
FROM (
      SELECT '{season1}{season2}' quota, '03' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season1}', '{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_suju_date >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_suju_date <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season1}{season2}' quota, '12' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season1}', '{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_appv_end_dt >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_appv_end_dt <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season2}' quota, '03' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_suju_date >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_suju_date <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season2}' quota, '12' gubun, Max(master_bokjong) bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t
       WHERE master_quota IN ('{season2}')
         AND master_jaepum = 'H'
         AND master_bokjong = 'J'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND master_appv_end_dt >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_appv_end_dt <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season3}' quota, '03' gubun, 'HA' bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t, i_cod_t tt
       WHERE master_quota IN ('{season3}')
         AND master_jaepum = 'H'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND tt.cod_code = master_bokjong
         AND tt.cod_gbn_code  = '01'
         AND nvl(tt.cod_etc2,'') = 'H'
         AND nvl(tt.cod_etc3,'') IN ('S','T')
         AND master_suju_date >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_suju_date <= to_date('{end_dt}','yyyy/mm/dd')
      UNION ALL
      SELECT '{season3}' quota, '12' gubun, 'HA' bokjong, Sum(master_suju_qty) qty
        FROM i_suju_master_t, i_cod_t tt
       WHERE master_quota IN ('{season3}')
         AND master_jaepum = 'H'
         AND master_tkyk IN ('C','D','H','I','L','R')
         AND master_status >= '03'
         AND master_status <= '60'
         AND tt.cod_code = master_bokjong
         AND tt.cod_gbn_code  = '01'
         AND nvl(tt.cod_etc2,'') = 'H'
         AND nvl(tt.cod_etc3,'') IN ('S','T')
         AND master_appv_end_dt >= to_date('{start_dt}','yyyy/mm/dd')
         AND master_appv_end_dt <= to_date('{end_dt}','yyyy/mm/dd')
         ) z
GROUP BY z.quota, z.gubun, z.bokjong
ORDER BY z.quota, z.gubun, z.bokjong desc
'''
    df = pd.read_sql_query(oracleSql, engine)
    df['gubun'] = df['gubun'].str.replace('03', '수주량').str.replace('12', '해제량')
    return df

In [46]:
df_suju_qty_week = suju_query_week(N_season, F_season, S_season, week_start, week_end)
df_suju_qty_week

,quota,gubun,bokjong,qty
0,22F,수주량,J,20.0
1,22F,해제량,J,91.0
2,22S,수주량,HA,NaN
3,22S,해제량,HA,NaN
4,23N22F,수주량,J,3360.0
5,23N22F,해제량,J,91.0


In [47]:
# sys.exit()

## 엑셀파일 작업 시작

### 엑셀기본 라이브러리

In [48]:
import xlwings as xw

### 환경변수

In [49]:
# file_path1 = "C:/Users/Administrator/PythonDataWorkspace/영업팀 일일자료/"
file_path2 = "C:/Users/Administrator/Documents/"

report_date2 = report_date[:4] + '-' + report_date[4:6] + '-' + report_date[6:]
report_date2, report_date

('2022-06-20', '20220620')

### '22년 학교주관구매 M.S 현황.xlsx 파일 오픈

#### 엑셀 인스턴스 생성

In [50]:
app = xw.App(visible=True) # 실행과정 보이게
# app = xw.App(visible=False) # 실행과정 안보이게

#### M.S 시트 지정

In [51]:
try:
    wb = xw.Book(f"./'22년 학교주관구매 M.S 현황.xlsx")
    sh_MS = wb.sheets['M.S']
except FileNotFoundError:
    print("같은 폴더안에 \n'22년 학교주관구매 M.S 현황.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    pyautogui.alert(title='FileNotFoundError', text="같은 폴더안에 \n'22년 학교주관구매 M.S 현황.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    sys.exit()

#### 동복 시트 지정

In [52]:
sh_NF = wb.sheets['# 동복']
sh_NF['J4:P20'].value = sh_NF['B4:H20'].value # 전일 데이터를 숨기기 된 컬럼에 값만 복사

#### 하복 시트 지정

In [53]:
sh_S = wb.sheets['# 하복']
sh_S['J4:P20'].value = sh_S['B4:H20'].value # 전일 데이터를 숨기기 된 컬럼에 값만 복사

#### 22F 시트 지정

In [54]:
sh_F = wb.sheets['# 22F']
sh_F['J4:P20'].value = sh_F['B4:H20'].value # 전일 데이터를 숨기기 된 컬럼에 값만 복사

#### 23N+22F -> M.S 시트에 기록

In [55]:
df_bid_tkyk_23N22F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,23N22F,C,1,서울상권,1,1,0,0,0,140,153,0,0,0
1,23N22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,23N22F,D,3,대전상권,1,1,0,0,0,149,184,0,0,0
3,23N22F,L,4,광주상권,49,49,39,48,60,7247,6688,5443,6678,8455
4,23N22F,I,5,대구상권,41,34,36,26,42,6426,5066,5709,4319,7752
5,23N22F,R,6,부산상권,22,7,10,8,17,2137,707,1225,1169,2974


In [56]:
# len(df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '서울상권'].index) == 1
df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '광주상권'].index[0]

3

In [57]:
sh_MS['H2'].value = report_date2

# 서울상권
if len(df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '서울상권'].index) == 1: # 서울상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '서울상권'].index[0] # 서울상권 인덱스번호

    # 한 줄 추가할 때는 시리즈로 자동변환 되어서 강제로 데이터프레임 변경 후 축을 돌려서 입력함
    sh_MS['D4'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D5'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D4'].value = [0, 0, 0, 0, 0]
    sh_MS['D5'].value = [0, 0, 0, 0, 0]


# 중부상권
if len(df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '중부상권'].index) == 1: # 중부상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '중부상권'].index[0] # 중부상권 인덱스번호
    sh_MS['D13'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D14'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D13'].value = [0, 0, 0, 0, 0]
    sh_MS['D14'].value = [0, 0, 0, 0, 0]


# 대전상권
if len(df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '대전상권'].index) == 1: # 대전상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '대전상권'].index[0] # 대전상권 인덱스번호
    sh_MS['D22'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D23'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D22'].value = [0, 0, 0, 0, 0]
    sh_MS['D23'].value = [0, 0, 0, 0, 0]

    
# 광주상권
if len(df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '광주상권'].index) == 1: # 광주상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '광주상권'].index[0] # 광주상권 인덱스번호
    sh_MS['D31'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D32'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D31'].value = [0, 0, 0, 0, 0]
    sh_MS['D32'].value = [0, 0, 0, 0, 0]


# 대구상권
if len(df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '대구상권'].index) == 1: # 대구상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '대구상권'].index[0] # 대구상권 인덱스번호
    sh_MS['D40'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D41'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D40'].value = [0, 0, 0, 0, 0]
    sh_MS['D41'].value = [0, 0, 0, 0, 0]


# 부산상권
if len(df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '부산상권'].index) == 1: # 부산상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_23N22F[df_bid_tkyk_23N22F['tkyk_name'] == '부산상권'].index[0] # 부산상권 인덱스번호
    sh_MS['D49'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['D50'].options(index=False, header=False).value = df_bid_tkyk_23N22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['D49'].value = [0, 0, 0, 0, 0]
    sh_MS['D50'].value = [0, 0, 0, 0, 0]

index_tmp = '' # 초기화

#### 22F -> M.S 시트에 기록

In [58]:
df_bid_tkyk_22F

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22F,C,1,서울상권,1,0,0,0,0,140,0,0,0,0
1,22F,H,2,중부상권,37,37,23,25,28,4762,5387,3353,4246,3790
2,22F,D,3,대전상권,0,0,0,0,0,0,0,0,0,0
3,22F,L,4,광주상권,47,49,38,47,59,6776,6688,5243,6422,8065
4,22F,I,5,대구상권,41,34,36,26,42,6426,5066,5709,4319,7752
5,22F,R,6,부산상권,22,7,10,8,16,2137,707,1225,1169,2818


In [59]:
# 서울상권
if len(df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '서울상권'].index) == 1: # 서울상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '서울상권'].index[0] # 서울상권 인덱스번호

    # 한 줄 추가할 때는 시리즈로 자동변환 되어서 강제로 데이터프레임 변경 후 축을 돌려서 입력함
    sh_MS['X4'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X5'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X4'].value = [0, 0, 0, 0, 0]
    sh_MS['X5'].value = [0, 0, 0, 0, 0]


# 중부상권
if len(df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '중부상권'].index) == 1: # 중부상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '중부상권'].index[0] # 중부상권 인덱스번호
    sh_MS['X13'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X14'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X13'].value = [0, 0, 0, 0, 0]
    sh_MS['X14'].value = [0, 0, 0, 0, 0]


# 대전상권
if len(df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '대전상권'].index) == 1: # 대전상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '대전상권'].index[0] # 대전상권 인덱스번호
    sh_MS['X22'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X23'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X22'].value = [0, 0, 0, 0, 0]
    sh_MS['X23'].value = [0, 0, 0, 0, 0]

    
# 광주상권
if len(df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '광주상권'].index) == 1: # 광주상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '광주상권'].index[0] # 광주상권 인덱스번호
    sh_MS['X31'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X32'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X31'].value = [0, 0, 0, 0, 0]
    sh_MS['X32'].value = [0, 0, 0, 0, 0]


# 대구상권
if len(df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '대구상권'].index) == 1: # 대구상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '대구상권'].index[0] # 대구상권 인덱스번호
    sh_MS['X40'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X41'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X40'].value = [0, 0, 0, 0, 0]
    sh_MS['X41'].value = [0, 0, 0, 0, 0]


# 부산상권
if len(df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '부산상권'].index) == 1: # 부산상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22F[df_bid_tkyk_22F['tkyk_name'] == '부산상권'].index[0] # 부산상권 인덱스번호
    sh_MS['X49'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['X50'].options(index=False, header=False).value = df_bid_tkyk_22F.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['X49'].value = [0, 0, 0, 0, 0]
    sh_MS['X50'].value = [0, 0, 0, 0, 0]

index_tmp = '' # 초기화

#### 22S -> M.S 시트에 기록

In [60]:
df_bid_tkyk_22S

,quota,tkyk,sort,tkyk_name,i_cnt,s_cnt,e_cnt,l_cnt,etc_cnt,i_qty,s_qty,e_qty,l_qty,etc_qty
0,22S,C,1,서울상권,171,167,127,127,386,33270,32474,24909,25254,76495
1,22S,H,2,중부상권,223,217,175,194,314,45731,48693,36576,43285,68225
2,22S,D,3,대전상권,161,125,109,95,153,22894,19628,16711,15137,20120
3,22S,L,4,광주상권,121,115,101,99,127,18088,17524,16023,15160,20081
4,22S,I,5,대구상권,103,103,73,70,96,18112,17639,12023,12083,17966
5,22S,R,6,부산상권,144,87,82,87,116,22038,14568,13419,14600,18879


In [61]:
# 서울상권
if len(df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '서울상권'].index) == 1: # 서울상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '서울상권'].index[0] # 서울상권 인덱스번호

    # 한 줄 추가할 때는 시리즈로 자동변환 되어서 강제로 데이터프레임 변경 후 축을 돌려서 입력함
    sh_MS['N4'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N5'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N4'].value = [0, 0, 0, 0, 0]
    sh_MS['N5'].value = [0, 0, 0, 0, 0]


# 중부상권
if len(df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '중부상권'].index) == 1: # 중부상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '중부상권'].index[0] # 중부상권 인덱스번호
    sh_MS['N13'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N14'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N13'].value = [0, 0, 0, 0, 0]
    sh_MS['N14'].value = [0, 0, 0, 0, 0]


# 대전상권
if len(df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '대전상권'].index) == 1: # 대전상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '대전상권'].index[0] # 대전상권 인덱스번호
    sh_MS['N22'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N23'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N22'].value = [0, 0, 0, 0, 0]
    sh_MS['N23'].value = [0, 0, 0, 0, 0]

    
# 광주상권
if len(df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '광주상권'].index) == 1: # 광주상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '광주상권'].index[0] # 광주상권 인덱스번호
    sh_MS['N31'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N32'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N31'].value = [0, 0, 0, 0, 0]
    sh_MS['N32'].value = [0, 0, 0, 0, 0]


# 대구상권
if len(df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '대구상권'].index) == 1: # 대구상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '대구상권'].index[0] # 대구상권 인덱스번호
    sh_MS['N40'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N41'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N40'].value = [0, 0, 0, 0, 0]
    sh_MS['N41'].value = [0, 0, 0, 0, 0]


# 부산상권
if len(df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '부산상권'].index) == 1: # 부산상권 row가 1줄 존재하는가?
    index_tmp = df_bid_tkyk_22S[df_bid_tkyk_22S['tkyk_name'] == '부산상권'].index[0] # 부산상권 인덱스번호
    sh_MS['N49'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_cnt':'etc_cnt'].to_frame().T
    sh_MS['N50'].options(index=False, header=False).value = df_bid_tkyk_22S.loc[index_tmp, 'i_qty':'etc_qty'].to_frame().T
else: # 조건을 만족하지 않을 때는 0으로 처리한다.
    sh_MS['N49'].value = [0, 0, 0, 0, 0]
    sh_MS['N50'].value = [0, 0, 0, 0, 0]

index_tmp = '' # 초기화

In [62]:
# sys.exit()

### '일일보고.xlsx 파일 오픈

In [63]:
try:
    wb_day = xw.Book(f"./'일일보고.xlsx")
    sh_yesterday = wb_day.sheets[-1]
except FileNotFoundError:
    print("같은 폴더안에 \n'일일보고.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    pyautogui.alert(title='FileNotFoundError', text="같은 폴더안에 \n'일일보고.xlsx \n파일이 없습니다.\n\n프로그램을 종료합니다.")
    sys.exit()

#### 마지막 시트복사

- copy (before=None, after=None, name=None)
- 매개변수 :
    - before ( sheet object , default None ) – 시트를 배치하려는 시트 개체 앞에
    - after ( sheet object , default None ) – 시트를 배치하려는 뒤에 오는 시트 객체, 기본적으로 모든 기존 시트 뒤에 배치됩니다.
    - name ( str , 기본값 None ) – 복사본의 시트 이름

In [64]:
try:
    sh_yesterday.copy(name=report_date[2:])
except ValueError:
    print(f'작성할 {report_date[2:]} 시트명이 이미 존재합니다.\n해당 시트를 삭제 후 프로그램을 다시 실행하세요.')
    pyautogui.alert(title='ValueError', text=f'작성할 {report_date[2:]} 시트명이 이미 존재합니다.\n해당 시트를 삭제 후 프로그램을 다시 실행하세요.')
    sys.exit()

#### 오늘 작성할 시트 지정 (복사된)

In [65]:
sh_today = wb_day.sheets[report_date[2:]]

#### 제목 -> 오늘 날짜

##### A1

In [66]:
title_sh = report_date[:4] + '.' + report_date[4:6] + '.' + report_date[6:] + ' 영업팀 일일보고'
sh_today['A1'].value = title_sh

#### 어제 데이터 영역 복사

##### F3

In [67]:
sh_today['F3'].value = sh_today['B3:E21'].value
sh_today['B3:E21'].value

[['04.19', None, None, None],
 ['22F+23N', "'23N", "'22S", "'22F"],
 [120.0, 2.0, 889.0, 118.0],
 [17623.0, 471.0, 156310.0, 17152.0],
 [0.231, 0.358, 0.206, 0.229],
 [91.0, 0.0, 775.0, 91.0],
 [13761.0, 0.0, 145671.0, 13761.0],
 [0.181, 0.0, 0.192, 0.184],
 [84.0, 1.0, 641.0, 83.0],
 [12810.0, 200.0, 116293.0, 12610.0],
 [0.168, 0.152, 0.153, 0.168],
 [85.0, 1.0, 649.0, 84.0],
 [13785.0, 256.0, 122835.0, 13529.0],
 [0.181, 0.194, 0.162, 0.181],
 [109.0, 1.0, 1152.0, 108.0],
 [18189.0, 390.0, 216733.0, 17799.0],
 [0.239, 0.296, 0.286, 0.238],
 [489.0, 5.0, 4106.0, 484.0],
 [76168.0, 1317.0, 757842.0, 74851.0]]

#### 오늘 데이터 영역 복사 (M.S파일에서) ***

- 복사하지 않고 직접 뽑아옴

##### B5

In [68]:
sh_today['B5'].options(index=False, header=False).value = df_ms_total
df_ms_total

quota,23N22F,23N,22S,22F
0,151.000,3.000,923.000,148.000
1,20861.000,620.000,160133.000,20241.000
2,0.222,0.316,0.206,0.220
3,129.000,2.000,814.000,127.000
4,18185.000,337.000,150526.000,17848.000
5,0.193,0.172,0.194,0.194
6,108.000,1.000,667.000,107.000
7,15730.000,200.000,119661.000,15530.000
8,0.167,0.102,0.154,0.168
9,107.000,1.000,672.000,106.000


#### 오늘 날짜 입력

##### B3

In [69]:
sh_today['B3'].value = report_date2
report_date2

'2022-06-20'

#### # 동복 -> 23N + 22F 낙찰현황 복사

##### B26

In [70]:
sh_today['B26'].options(index=False, header=False).value = df_bid_tkyk_23N22F_ISEL_sum
df_bid_tkyk_23N22F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,1,140,1,153,0,0,0,0,2,293
1,37,4762,37,5387,23,3353,25,4246,122,17748
2,1,149,1,184,0,0,0,0,2,333
3,49,7247,49,6688,39,5443,48,6678,185,26056
4,41,6426,34,5066,36,5709,26,4319,137,21520
5,22,2137,7,707,10,1225,8,1169,47,5238
6,151,20861,129,18185,108,15730,107,16412,495,71188


#### # 하복 -> 22S 낙찰현황 복사

##### B48

In [71]:
sh_today['B48'].options(index=False, header=False).value = df_bid_tkyk_22S_ISEL_sum
df_bid_tkyk_22S_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty,sum_cnt,sum_qty
0,171,33270,167,32474,127,24909,127,25254,592,115907
1,223,45731,217,48693,175,36576,194,43285,809,174285
2,161,22894,125,19628,109,16711,95,15137,490,74370
3,121,18088,115,17524,101,16023,99,15160,436,66795
4,103,18112,103,17639,73,12023,70,12083,349,59857
5,144,22038,87,14568,82,13419,87,14600,400,64625
6,923,160133,814,150526,667,119661,672,125519,3076,555839


#### # 22F -> 22F 낙찰현황 복사

##### B59

In [72]:
sh_today['B59'].options(index=False, header=False).value = df_bid_tkyk_22F_ISEL_sum
df_bid_tkyk_22F_ISEL_sum

,i_cnt,i_qty,s_cnt,s_qty,e_cnt,e_qty,l_cnt,l_qty
0,1,140,0,0,0,0,0,0
1,37,4762,37,5387,23,3353,25,4246
2,0,0,0,0,0,0,0,0
3,47,6776,49,6688,38,5243,47,6422
4,41,6426,34,5066,36,5709,26,4319
5,22,2137,7,707,10,1225,8,1169
6,148,20241,127,17848,107,15530,106,16156


#### # 수주 전일데이터 복사

##### E90, E103, E115

In [73]:
sh_today['E90'].value = sh_today['B90:D96'].value # 23년 동복(22F+23N) 수주 현황 ( J + H 기준 )
sh_today['E103'].value = sh_today['B103:D109'].value # 22년 하복(22S) 수주 현황 ( 하의기준 )
sh_today['E115'].value = sh_today['B115:D121'].value # 23년 동복(22F) 수주 현황 ( J + H 기준 )

In [74]:
# sys.exit()

## 데이터 프레임 작성

### 기록할 포지션 잡기 (상대좌표)

In [75]:
# 범위 안에 원하는 TEXT 값 찾기
i = 0
set_position = [] # 가변 좌표

for row in range(1, 200): # row 범위
    for col in range(1, 2): # column 범위
        if sh_today.range((row, col)).value == "서울":
            i += 1
            if i > 4: # 5번째 부터 찾는다.
                print(f'{col}열 {row}행')
                set_position.append(row)
                print(set_position)
                
if len(set_position) != 3:
    print(f"기준좌표 갯수가 3 이어야 합니다.\n현재값 : {len(set_position)}\n\n프로그램을 종료합니다.")
    pyautogui.alert(title='논리 에러', text=f"기준좌표 갯수가 3 이어야 합니다.\n현재값 : {len(set_position)}\n\n프로그램을 종료합니다.")
    sys.exit()

1열 86행
[86]
1열 99행
[86, 99]
1열 111행
[86, 99, 111]


### 시즌, 상권별 수주/해제량

#### NF, 수주량/해제량, J+H

In [76]:
cond1 = df_suju_qty['quota'] == '23N22F'
cond2 = df_suju_qty['gubun'] == '수주량'
cond3 = df_suju_qty['gubun'] == '해제량'

df_NF_JH_SUJU = df_suju_qty[cond1 & cond2]
df_NF_JH_HAJE = df_suju_qty[cond1 & cond3]

df_NF_JH_SUJU, df_NF_JH_HAJE

(     quota gubun bokjong tkyk tkyk_name  sort   qty
 32  23N22F   수주량       J    C      서울상권     1   115
 33  23N22F   수주량       J    H      중부상권     2  4652
 34  23N22F   수주량       J    D      대전상권     3  1464
 35  23N22F   수주량       J    L      광주상권     4  3403
 36  23N22F   수주량       J    I      대구상권     5  5439
 37  23N22F   수주량       J    R      부산상권     6  4201
 38  23N22F   수주량       H    H      중부상권     2    22
 39  23N22F   수주량       H    D      대전상권     3  1340
 40  23N22F   수주량       H    L      광주상권     4  1567
 41  23N22F   수주량       H    I      대구상권     5  1459,
      quota gubun bokjong tkyk tkyk_name  sort   qty
 42  23N22F   해제량       J    C      서울상권     1   115
 43  23N22F   해제량       J    H      중부상권     2  2800
 44  23N22F   해제량       J    D      대전상권     3   294
 45  23N22F   해제량       J    L      광주상권     4  2287
 46  23N22F   해제량       J    I      대구상권     5  3211
 47  23N22F   해제량       J    R      부산상권     6  1209
 48  23N22F   해제량       H    H      중부상권     

In [77]:
df_NF_JH = pd.concat([df_NF_JH_SUJU.groupby(['sort', 'tkyk'])['qty'].agg(sum),
                      df_NF_JH_HAJE.groupby(['sort', 'tkyk'])['qty'].agg(sum)
                     ], axis=1)
df_NF_JH.columns = ['수주량', '해제량']

df_NF_JH.loc[str(len(df_NF_JH)+1)] = df_NF_JH.sum() # 합계. 인덱스 꼬임. 그냥 진행.

df_NF_JH['해제율'] = round(df_NF_JH['해제량'] / df_NF_JH['수주량'], 3)

df_NF_JH

,수주량,해제량,해제율
"(1, C)",115,115,1.000
"(2, H)",4674,2822,0.604
"(3, D)",2804,372,0.133
"(4, L)",4970,3494,0.703
"(5, I)",6898,3798,0.551
"(6, R)",4201,1209,0.288
7,23662,11810,0.499


##### 포지션 1

In [78]:
sh_today[f'B{set_position[0]}'].options(index=False, header=False).value = df_NF_JH
df_NF_JH

,수주량,해제량,해제율
"(1, C)",115,115,1.000
"(2, H)",4674,2822,0.604
"(3, D)",2804,372,0.133
"(4, L)",4970,3494,0.703
"(5, I)",6898,3798,0.551
"(6, R)",4201,1209,0.288
7,23662,11810,0.499


#### S, 수주량/해제량, 하의

In [79]:
cond1 = df_suju_qty['quota'] == '22S'
cond2 = df_suju_qty['gubun'] == '수주량'
cond3 = df_suju_qty['gubun'] == '해제량'

df_S_HAI_SUJU = df_suju_qty[cond1 & cond2]
df_S_HAI_HAJE = df_suju_qty[cond1 & cond3]

df_S_HAI_SUJU, df_S_HAI_HAJE

(   quota gubun bokjong tkyk tkyk_name  sort    qty
 20   22S   수주량      HA    C      서울상권     1  46167
 21   22S   수주량      HA    H      중부상권     2  56242
 22   22S   수주량      HA    D      대전상권     3  23915
 23   22S   수주량      HA    L      광주상권     4  27495
 24   22S   수주량      HA    I      대구상권     5  28031
 25   22S   수주량      HA    R      부산상권     6  30930,
    quota gubun bokjong tkyk tkyk_name  sort    qty
 26   22S   해제량      HA    C      서울상권     1  46167
 27   22S   해제량      HA    H      중부상권     2  56242
 28   22S   해제량      HA    D      대전상권     3  23915
 29   22S   해제량      HA    L      광주상권     4  27495
 30   22S   해제량      HA    I      대구상권     5  28031
 31   22S   해제량      HA    R      부산상권     6  30930)

In [80]:
df_S_HAI = pd.concat([df_S_HAI_SUJU.groupby(['sort', 'tkyk'])['qty'].agg(sum),
                      df_S_HAI_HAJE.groupby(['sort', 'tkyk'])['qty'].agg(sum)
                     ], axis=1)
df_S_HAI.columns = ['수주량', '해제량']

df_S_HAI.loc[str(len(df_S_HAI)+1)] = df_S_HAI.sum() # 합계. 인덱스 꼬임. 그냥 진행.

df_S_HAI['해제율'] = round(df_S_HAI['해제량'] / df_S_HAI['수주량'], 3)

df_S_HAI

,수주량,해제량,해제율
"(1, C)",46167,46167,1.0
"(2, H)",56242,56242,1.0
"(3, D)",23915,23915,1.0
"(4, L)",27495,27495,1.0
"(5, I)",28031,28031,1.0
"(6, R)",30930,30930,1.0
7,212780,212780,1.0


##### 포지션 2

In [81]:
sh_today[f'B{set_position[1]}'].options(index=False, header=False).value = df_S_HAI
df_S_HAI

,수주량,해제량,해제율
"(1, C)",46167,46167,1.0
"(2, H)",56242,56242,1.0
"(3, D)",23915,23915,1.0
"(4, L)",27495,27495,1.0
"(5, I)",28031,28031,1.0
"(6, R)",30930,30930,1.0
7,212780,212780,1.0


#### F, 수주량/해제량, J+H

In [82]:
cond1 = df_suju_qty['quota'] == '22F'
cond2 = df_suju_qty['gubun'] == '수주량'
cond3 = df_suju_qty['gubun'] == '해제량'

df_F_JH_SUJU = df_suju_qty[cond1 & cond2]
df_F_JH_HAJE = df_suju_qty[cond1 & cond3]

df_F_JH_SUJU, df_F_JH_HAJE

(  quota gubun bokjong tkyk tkyk_name  sort   qty
 0   22F   수주량       J    C      서울상권     1   115
 1   22F   수주량       J    H      중부상권     2  4652
 2   22F   수주량       J    D      대전상권     3    54
 3   22F   수주량       J    L      광주상권     4  3403
 4   22F   수주량       J    I      대구상권     5  4643
 5   22F   수주량       J    R      부산상권     6  1849
 6   22F   수주량       H    H      중부상권     2    22
 7   22F   수주량       H    D      대전상권     3    10
 8   22F   수주량       H    L      광주상권     4  1567
 9   22F   수주량       H    I      대구상권     5  1459,
    quota gubun bokjong tkyk tkyk_name  sort   qty
 10   22F   해제량       J    C      서울상권     1   115
 11   22F   해제량       J    H      중부상권     2  2800
 12   22F   해제량       J    D      대전상권     3    54
 13   22F   해제량       J    L      광주상권     4  2287
 14   22F   해제량       J    I      대구상권     5  2475
 15   22F   해제량       J    R      부산상권     6  1027
 16   22F   해제량       H    H      중부상권     2    22
 17   22F   해제량       H    D      대전상권   

In [83]:
df_F_JH = pd.concat([df_F_JH_SUJU.groupby(['sort', 'tkyk'])['qty'].agg(sum),
                     df_F_JH_HAJE.groupby(['sort', 'tkyk'])['qty'].agg(sum)
                    ], axis=1)
df_F_JH.columns = ['수주량', '해제량']

df_F_JH.loc[str(len(df_F_JH)+1)] = df_F_JH.sum() # 합계. 인덱스 꼬임. 그냥 진행.

df_F_JH['해제율'] = round(df_F_JH['해제량'] / df_F_JH['수주량'], 3)

df_F_JH

,수주량,해제량,해제율
"(1, C)",115,115,1.000
"(2, H)",4674,2822,0.604
"(3, D)",64,64,1.000
"(4, L)",4970,3494,0.703
"(5, I)",6102,3062,0.502
"(6, R)",1849,1027,0.555
7,17774,10584,0.595


##### 포지션 3

In [84]:
sh_today[f'B{set_position[2]}'].options(index=False, header=False).value = df_F_JH
df_F_JH

,수주량,해제량,해제율
"(1, C)",115,115,1.000
"(2, H)",4674,2822,0.604
"(3, D)",64,64,1.000
"(4, L)",4970,3494,0.703
"(5, I)",6102,3062,0.502
"(6, R)",1849,1027,0.555
7,17774,10584,0.595


In [85]:
# sys.exit()

#### 시즌, 수주/해제, 복종별 집계

- df.groupby([열1,열2...열n])[[열1,열2...열n]].agg("집계함수").reset_index()
- df.pivot_table(values열, index=[열1,열2...열n], columns=[열1,열2...열n], aggfunc="집계함수")

In [86]:
df_suju_qty_sum = df_suju_qty.groupby(['quota', 'gubun', 'bokjong'])['qty'].agg(sum).to_frame()
df_suju_qty_sum = df_suju_qty_sum.reset_index()
df_suju_qty_sum

,quota,gubun,bokjong,qty
0,22F,수주량,H,3058
1,22F,수주량,J,14716
2,22F,해제량,H,1826
3,22F,해제량,J,8758
4,22S,수주량,HA,212780
5,22S,해제량,HA,212780
6,23N22F,수주량,H,4388
7,23N22F,수주량,J,19274
8,23N22F,해제량,H,1894
9,23N22F,해제량,J,9916


In [87]:
sum_quota = df_suju_qty_sum['quota'].unique()
sum_gubun = df_suju_qty_sum['gubun'].unique()
sum_bokjong = df_suju_qty_sum['bokjong'].unique()

cond1_1 = df_suju_qty_sum['quota'] == sum_quota[2]
# cond1_2 = df_suju_qty_sum['quota'] == sum_quota[1]
cond1_3 = df_suju_qty_sum['quota'] == sum_quota[0]
cond2_1 = df_suju_qty_sum['gubun'] == sum_gubun[0]
cond2_2 = df_suju_qty_sum['gubun'] == sum_gubun[1]
cond3_1 = df_suju_qty_sum['bokjong'] == sum_bokjong[1]
cond3_2 = df_suju_qty_sum['bokjong'] == sum_bokjong[0]
# cond3_3 = df_suju_qty_sum['bokjong'] == sum_bokjong[2]

# 23N22F
NF_SUJU_J = df_suju_qty_sum['qty'][cond1_1 & cond2_1 & cond3_1].sum()
NF_SUJU_H = df_suju_qty_sum['qty'][cond1_1 & cond2_1 & cond3_2].sum()
NF_HAJE_J = df_suju_qty_sum['qty'][cond1_1 & cond2_2 & cond3_1].sum()
NF_HAJE_H = df_suju_qty_sum['qty'][cond1_1 & cond2_2 & cond3_2].sum()

# 22S (사용안함)
# S_SUJU_HA = df_suju_qty_sum['qty'][cond1_2 & cond2_1 & cond3_3].sum()
# S_HAJE_HA = df_suju_qty_sum['qty'][cond1_2 & cond2_2 & cond3_3].sum()

# 22F
F_SUJU_J = df_suju_qty_sum['qty'][cond1_3 & cond2_1 & cond3_1].sum()
F_SUJU_H = df_suju_qty_sum['qty'][cond1_3 & cond2_1 & cond3_2].sum()
F_HAJE_J = df_suju_qty_sum['qty'][cond1_3 & cond2_2 & cond3_1].sum()
F_HAJE_H = df_suju_qty_sum['qty'][cond1_3 & cond2_2 & cond3_2].sum()

NF_SUJU_J, NF_SUJU_H, NF_HAJE_J, NF_HAJE_H, F_SUJU_J, F_SUJU_H, F_HAJE_J, F_HAJE_H

(19274, 4388, 9916, 1894, 14716, 3058, 8758, 1826)

##### (포지션 1) + 7

In [88]:
sh_today[f'A{set_position[0]+7}'].value = f'수주량 J : {NF_SUJU_J:,} ,  H : {NF_SUJU_H:,} / 해제량 J : {NF_HAJE_J:,} ,  H : {NF_HAJE_H:,}'
print(f'수주량 J : {NF_SUJU_J:,} ,  H : {NF_SUJU_H:,} / 해제량 J : {NF_HAJE_J:,} ,  H : {NF_HAJE_H:,}')

수주량 J : 19,274 ,  H : 4,388 / 해제량 J : 9,916 ,  H : 1,894


##### (포지션 3) + 7

In [89]:
sh_today[f'A{set_position[2]+7}'].value = f'수주량 J : {F_SUJU_J:,} ,  H : {F_SUJU_H:,} / 해제량 J : {F_HAJE_J:,} ,  H : {F_HAJE_H:,}'
print(f'수주량 J : {F_SUJU_J:,} ,  H : {F_SUJU_H:,} / 해제량 J : {F_HAJE_J:,} ,  H : {F_HAJE_H:,}')

수주량 J : 14,716 ,  H : 3,058 / 해제량 J : 8,758 ,  H : 1,826


#### 전주간 수주/해제량

In [90]:
sum_quota_week = df_suju_qty_week['quota'].unique()
sum_gubun_week = df_suju_qty_week['gubun'].unique()
sum_bokjong_week = df_suju_qty_week['bokjong'].unique()

cond1_1 = df_suju_qty_week['quota'] == sum_quota_week[2]
cond1_2 = df_suju_qty_week['quota'] == sum_quota_week[1]
cond1_3 = df_suju_qty_week['quota'] == sum_quota_week[0]
cond2_1 = df_suju_qty_week['gubun'] == sum_gubun_week[0]
cond2_2 = df_suju_qty_week['gubun'] == sum_gubun_week[1]
cond3_1 = df_suju_qty_week['bokjong'] == sum_bokjong_week[0]
cond3_2 = df_suju_qty_week['bokjong'] == sum_bokjong_week[1]

# 23N22F
NF_SUJU_J_WEEK = df_suju_qty_week['qty'][cond1_1 & cond2_1 & cond3_1].sum()
NF_HAJE_J_WEEK = df_suju_qty_week['qty'][cond1_1 & cond2_2 & cond3_1].sum()

# 22S
S_SUJU_HA_WEEK = df_suju_qty_week['qty'][cond1_2 & cond2_1 & cond3_2].sum()
S_HAJE_HA_WEEK = df_suju_qty_week['qty'][cond1_2 & cond2_2 & cond3_2].sum()

# 22F
F_SUJU_J_WEEK = df_suju_qty_week['qty'][cond1_3 & cond2_1 & cond3_1].sum()
F_HAJE_J_WEEK = df_suju_qty_week['qty'][cond1_3 & cond2_2 & cond3_1].sum()

NF_SUJU_J_WEEK, NF_HAJE_J_WEEK, S_SUJU_HA_WEEK, S_HAJE_HA_WEEK, F_SUJU_J_WEEK, F_HAJE_J_WEEK

(3360.0, 91.0, 0.0, 0.0, 20.0, 91.0)

##### (포지션 1-3) + 7-9

In [91]:
sh_today[f'A{set_position[0]+8}'].value = f'前주간 J 수주량 : {NF_SUJU_J_WEEK:,}'
sh_today[f'A{set_position[0]+9}'].value = f'前주간 J 해제량 : {NF_HAJE_J_WEEK:,}'

sh_today[f'A{set_position[1]+7}'].value = f'前주간 수주량 : {S_SUJU_HA_WEEK:,}'
sh_today[f'A{set_position[1]+8}'].value = f'前주간 해제량 : {S_HAJE_HA_WEEK:,}'

sh_today[f'A{set_position[2]+8}'].value = f'前주간 J 수주량 : {F_SUJU_J_WEEK:,}'
sh_today[f'A{set_position[2]+9}'].value = f'前주간 J 해제량 : {F_HAJE_J_WEEK:,}'

print(f'前주간 J 수주량 : {NF_SUJU_J_WEEK:,}')
print(f'前주간 J 해제량 : {NF_HAJE_J_WEEK:,}')
print(f'前주간 수주량 : {S_SUJU_HA_WEEK:,}')
print(f'前주간 해제량 : {S_HAJE_HA_WEEK:,}')
print(f'前주간 J 수주량 : {F_SUJU_J_WEEK:,}')
print(f'前주간 J 해제량 : {F_HAJE_J_WEEK:,}')

前주간 J 수주량 : 3,360.0
前주간 J 해제량 : 91.0
前주간 수주량 : 0.0
前주간 해제량 : 0.0
前주간 J 수주량 : 20.0
前주간 J 해제량 : 91.0


In [92]:
# sys.exit()

### 파일저장 및 종료

In [93]:
wb.save()

In [94]:
wb.close()

In [95]:
wb_day.save()

In [96]:
wb_day.close()

## 작업종료

In [97]:
app.kill()

### 수행시간측정 종료

In [98]:
end_lab = timer()

In [99]:
pyautogui.alert(title='수행시간 측정', text=f'시작시각 : {start_lab}\n종료시각 : {end_lab}\n\n총 수행시간 : {end_lab-start_lab} 초')

'OK'

In [100]:
pyautogui.alert(title='주의사항', text='수치데이터 이외에 코멘트 기입은 수기로 작성해주시기 바랍니다.\n엑셀양식 표 위치를 변경하면 정상동작하지 않습니다.\n표 위치를 변경하면 프로그램을 수정해야 합니다.')

'OK'

In [101]:
pyautogui.alert(title='프로그램 완료', text='지루한 작업이 모두 끝났습니다. \n수고하셨습니다.')

'OK'